# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the same account you used in the lab (hvt201)  and query the database that you were querying in class (citibike_feb_2015)

In [1]:
# SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql?q='
SQL_SOURCE = 'https://andrewnell.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (andrewnell Carto account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-574420f04fd4>, line 24)

# TASK 1

In [ ]:
# Sorting, Filtering and top 10 rows
task1_SortTop10 = '''
SELECT *
FROM citibike 
WHERE tripduration >= (3*3600)
ORDER BY start_station_id, tripduration
LIMIT 10 
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_SortTop10)), sep=',').head()

In [ ]:
# Sorting, filtering and bottom 10 rows
task1_SortBottom10 = '''
SELECT *
FROM citibike 
WHERE tripduration >= (3*3600)
ORDER BY start_station_id DESC, tripduration DESC
LIMIT 10 
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_SortBottom10)), sep=',').head()

In [ ]:
# Get Unique start station IDs
task1_UniqueStart = '''
SELECT DISTINCT start_station_id
FROM citibike
ORDER BY start_station_id
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_UniqueStart)), sep=',').head()

In [ ]:
# Get Count, Max, Min and Average trip duration
task1_CMMA = '''
SELECT COUNT(*),MAX(tripduration),MIN(tripduration), AVG(tripduration) 
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_CMMA)), sep=',')

# TASK 2
### Define SQL Functions

In [ ]:
# Select Trips that only start on 2015/02/02
Task2_20150202 = '''
SELECT *
FROM citibike
WHERE CAST(starttime AS Date)  = '2015-02-02'
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task2_20150202)), sep=',').head()

In [ ]:
# Select Average Weekend Trip Duration
Task2_AvgWeekend = '''
SELECT AVG(tripduration)
FROM citibike
WHERE EXTRACT(DOW FROM starttime) IN (0,6)
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task2_AvgWeekend)), sep=',').head()

In [ ]:
# Select Average Weekday Trip Duration
Task2_AvgWeekday = '''
SELECT AVG(tripduration)
FROM citibike
WHERE EXTRACT(DOW FROM starttime) NOT IN (0,6)
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task2_AvgWeekday)), sep=',').head()

# TASK 3

In [ ]:
# List of start locations using Group by
Task3_StartLoc = '''
SELECT start_station_name
FROM citibike
GROUP BY start_station_name
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task3_StartLoc)), sep=',').head()

In [ ]:
# Count of start trips by start locations using Group by
Task3_TripsStartLoc = '''
SELECT start_station_name, COUNT(tripduration)
FROM citibike
GROUP BY start_station_name
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task3_TripsStartLoc)), sep=',').head()

In [ ]:
# Count of start trips by start locations using Group by within 500 meters of Time Square
Task3_TripsStartLocTS = '''
SELECT start_station_name, COUNT(tripduration)
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                            start_station_longitude
                           ) :: geography,
                 CDB_LatLng(40.7577, -73.9857)  :: geography, 500)
GROUP BY start_station_name
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task3_TripsStartLocTS)), sep=',').head()

# TASK 4

In [ ]:
# Max Average Trip Duration by start locations within 500 meters of Time Square
Task4_1 = '''
SELECT start_station_name, AVG(tripduration)
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                            start_station_longitude
                           ) :: geography,
                 CDB_LatLng(40.7577, -73.9857)  :: geography, 500) 
      AND EXTRACT(DOW FROM starttime) NOT IN (0,6)
GROUP BY start_station_name
ORDER BY AVG(tripduration) DESC
LIMIT 1
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task4_1)), sep=',')

In [ ]:
# Lines of Trips within 500 meters of Time Square below 2 hours
Task4_2 = '''
SELECT ST_Makeline(
                   CDB_TransformToWebmercator(
                                              CDB_LatLng(
                                                    start_station_latitude,
                                                    start_station_longitude
                                                        )
                                             ),
                   CDB_TransformToWebmercator(
                                              CDB_LatLng(
                                                    end_station_latitude,
                                                    end_station_longitude
                                                        )
                                             )
                   )
                   as the_geom_webmercator,
                   MIN(cartodb_id) as cartodb_id,
                   COUNT(cartodb_id) as number_trips
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                            start_station_longitude
                           ) :: geography,
                 CDB_LatLng(40.7577, -73.9857)  :: geography, 500) 
      AND tripduration <= (2*3600)
GROUP BY start_station_id, start_station_latitude, start_station_longitude,
                 end_station_id, end_station_latitude, end_station_longitude
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task4_2)), sep=',').head()

# extra credit: make the function python 2 and 3 compatible so that it works on the  PUI2016_Python3 kernel

## ASH - Score 100. Extra Credit 0